In [ ]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme()
sns.set_style("whitegrid")
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading in the data

In [ ]:

csv_files = glob.glob('/kaggle/input/alcohol-and-drug-treatment-services-australia/*.csv')

combined_df = pd.DataFrame()

for file in csv_files:
    df = pd.read_csv(file, dtype = str)
    combined_df = pd.concat([combined_df, df], axis = 0)
    
combined_df.info()

As you can see, there are some issues with column naming, such that two columns that contain the same information are not named similarly. 

I will have to import each CSV file manually, and change the column names
There should be 18 colums in total

In [ ]:
# import each file separately

df1 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/09d5c21d-a992-43f1-bb72-5753a5e2210b.csv')
df2 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/4c24f305-af89-4649-83ed-f84ddb16bdde.csv')
df3 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/4df60a66-9558-41c6-92e1-af70a06232a0.csv')
df4 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/9196a14d-f6d1-4781-8995-83d027b8c084.csv')
df5 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/c0c1888c-1be8-4de1-b757-ca3e67423e1a.csv')
df6 = pd.read_csv('/kaggle/input/alcohol-and-drug-treatment-services-australia/dec48274-a9d2-457b-a79a-8db485d7ddb7.csv')

In [ ]:
# brief check to look at column names

df1.info()
df2.info()

In [ ]:
# get a list of column names from the first dataframe to be used for all other dataframes.
# do some additional work to lowercase and underscore them for ease.

column_names = df1.columns
column_names = [x.lower() for x in column_names]
column_names = [w.replace(" ", "_") for w in column_names]
print(column_names)

In [ ]:
# rename all dataframes to have identical column names and check.

dataframes = [df1,df2,df3,df4,df5,df6]
for dataframe in dataframes:
    dataframe.columns = column_names
    
df1.info()
df2.info()

We should now have matching column names for all dataframes, so can concatenate them together

In [ ]:
data = pd.DataFrame()

for dataframe in dataframes:
    data = pd.concat([data, dataframe], ignore_index = True)
    
data.info()

# A little bit of data cleaning

In [ ]:
data.head(10)

The columns for which i need to find some keys:
 - Sex (done)
 - Indigenous Group
 - Clent Type
 - SoR groupings
 - Rfc grouping
 - Treatment delivery setting
 - injecting drug use
 - principal drug of concern
 - main treatment type
 - accom groupings

In [ ]:
variables = ['sex', 'indigenous_group', 'client_type', 'sor_groupings', 'rfc_grouping', 'treatment_delivery_setting', 'injecting_drug_use', 'principal_drug_of_concern', 'main_treatment_type', 'accom_groupings']
for variable in variables:
    print(data[variable].value_counts())

In [ ]:
# lets drop client type as they are all 'own use'
data.drop(columns = 'client_type', inplace = True)

In [ ]:
# create a dictionary of values for each column. These were found online.

dict = {
    "indigenous_group": {1: "aboriginal", 2: "torres strait", 9: "na"},
    "sor_groupings": {1: "self", 2: "family/friend", 3: "medical practitioner", 9: "police"},
    "rfc_grouping": {1: 'completed', 2: "change in main treatment type", 3: "change in delivery setting", 9: "ceased to participate at expiation"},
    "treatment_delivery_setting": {1: "non-residential facility", 2: "residential facility", 3:"home", 4: "outreach setting", 8: "other"},
    "injecting_drug_use": {1: "=< 3 months", 2: "3-12 months", 3: ">12 months", 4: "never injected", 9: "na"},
    "main_treatment_type": {1: "withdrawal management", 2: "counselling", 3: "rehabiliation", 4: "pharmacotherapy", 5: "support and case management", 6: "info and education", 7: "assessment only", 8: "other", 88: "other"},
    "accom_groupings": {1: "independent residence", 2: "supported living", 3: "medical residence", 4: "other"},
    "age_group": {'Not stated': 'Not Stated'}
        }

In [ ]:
# replace the values in the original dataframe for each column with the values listed in the dictionary.

data.replace(to_replace = dict, inplace = True)
data.head()

I still need to replace the principal drug of concern. 

My idea here is to download in the data entry csv, create a dictionary based on the drug codes, and then replace.

In [ ]:
# download additional drug lookup csv that contains the drug codes and the respective drug.

drug_code = pd.read_csv("/kaggle/input/drug-code/drug_lookup.csv")
drug_code.head()

In [ ]:
# Use regex to get the four letter code from drug lookup and create a new dataframe with a column for code and a column for drug.

regex_pattern = r'(?<=\d\d\d\d)\s.\s'
drug_code[['code', 'drug']] = drug_code["Drug_and_code"].str.split(regex_pattern, regex = True, expand = True)
drug_code.drop(columns = ['Drug_and_code'], inplace = True)
drug_code.drop_duplicates(subset = ['code'], inplace = True)
drug_code.head(10)

In [ ]:
# create a dictionary using that dataframe that can then be used to replace the column.

drug_code['code'] = pd.to_numeric(drug_code['code'])
drug_dictionary = drug_code.set_index('code')['drug'].to_dict()
data['principal_drug_of_concern'].replace(to_replace = drug_dictionary, inplace = True)
data.head(10)

Now i need to extract the treatment start(day, month, and year) and end dates(day, month, and year) from month of commencement/cessation.

In [ ]:
# Convert the columns to datetime
data['month_of_commencement'] = pd.to_datetime(data['month_of_commencement'], format = 'mixed')
data['month_of_cessation'] = pd.to_datetime(data['month_of_cessation'], format = 'mixed')

# and then extract the day, month, year, from each.
data['start_year'] = data['month_of_commencement'].dt.year
data['start_month'] = data['month_of_commencement'].dt.month_name()
data['start_day'] = data['month_of_commencement'].dt.day
data['end_year'] = data['month_of_cessation'].dt.year
data['end_month'] = data['month_of_cessation'].dt.month_name()
data['end_day'] = data['month_of_cessation'].dt.day
data.head()

a couple of potential questions to look at. 
 - What predicts someone finishing treatment?
 - What drugs have the longest treatment?
 - what predicts someone dropping out?
 - do Private sectors offer better treatment than public?
 - Why are some people likely to come in more than once?


In [ ]:
data['completed'] = data['rfc_grouping'].apply(lambda x: 1 if x == 'completed' else 0) # create a new column based on if they completed the treatment.
data['visit_number'] = data.groupby('_id').cumcount() + 1 # create a new column that tracks what visit number it is for the person.
data['more_than_one_visit'] = data['visit_number'].apply(lambda x: 1 if x > 1 else 0) # create new column that indicates if the person has had more than one visit.

In [ ]:
# Lets just check everything is ok still
data.info()

In [ ]:
# and finally lets convert a few of the columns to more appropriate data types
# whether converting date to categorical is appropriate or not is not the focus here.
category = 'age_group'
object = ['sex', 'completed', 'more_than_one_visit', 'start_year', 'start_month', 'start_day', 'end_year', 'end_month', 'end_day', 'visit_number']
data[category] = data[category].astype('category')
data[object] = data[object].astype('object')

In [ ]:
# lets look at the different type of drugs used, to group them into categories later for visualisation and analysis.
for col in df:
    print(data['principal_drug_of_concern'].unique())


In [ ]:
# Create a dictionary with those using the new category as a key and the old values as values.
# I appreciate this is the wrong way around when it comes to use .replace, so I will need to do a bit extra.
# I also appreciate that this likely isn't 100% accurate. For instance, i have not included 'dissociatives' as a category.
drug_cat = {
    'cannabis': ['cannabis', 'Cannabinoids and Related Drugs', 'Cannabinoid agonists�', 'Cannabinoids and Related Drugs, nec', 'Cannabis'],
    'cns stimulant': ['Amphetamine', 'Methamphetamine', 'Nicotine', 'Cocaine', 'Herbal ecstasy', 'Ecstasy', 'Dexamphetamine', 'Methamphetamine analogues', 'Amphetamines, nec', 'Adipex P',
                      '3,4-methylenedioxyamphetamine', 'Cathine', 'Ephedrine', '1-Benzylpiperazine', '3,4-methylendioxyethamphetamine'] ,
    'cns depressant': ['Alcohol', 'Benzodizepines, nec', 'Euhypnos', 'Diazepam', 'Sleepers', 'Imovane', 'Alprazolam', 'Other sedatives and hypnotics, nec', 'Alcohols, nec', 'Quetiapine',
                       'Gamma-hydroxybutyrate', 'Atosil', 'Methylphenidate', 'Methanol', 'Alepam', 'Clonazepam', 'Amitriptyline', 'GHB type drugs and analogues, nec', 'Olanzapine', 'Orap',
                       '1,4-butanediol', 'Antidepressants', 'Clopine', 'Efexor', 'Psychostimulants, nec', 'Downers', 'Phenobarbitone', 'Barbiturates, nec', 'Desipramine hydrochloride', 'Ativan',
                       'Adormix', 'Sertraline', 'Cipramil', 'SARI', 'Chlorpromazine', 'Alodorm', 'Asenapine', 'Apo-Risperidone', 'Ketamine'],
    'hallucinogen': ['Acid', 'Blue meanies', '3-(2-dimethylaminoethyl)-4hydroxyindole', 'Angel dust', '3,4,5-trimethoxyphenylethylamine', '2,5-dimethoxy-4-ethyl-a-amphetamine', 'DET',],
    'analgesics': ['Oxycodone', 'Heroin', 'Opioids, nec', 'Codeine', 'Concentrate of poppy straw', 'Larapam SR', 'Dihydrocodeine', 'Non Opioid Analgesics, nec', 'Morphine', 'Methadone',
                   'Buprenorphine', 'Fentanyl', 'Alphaprodine', 'Opiod Antagonists, nec', 'Analgesics, nfd', 'Cafergot', 'Advil', 'Paracetamol', 'Isonipecaine', '3-methylfentanyl', 'Aerrane',
                   'LAAM'],
    'inhalant': ['Nitrous oxide', 'Butane', 'Amyl','Airplane glue', 'Cyclohexyl nitrate', "Businessman's lunch", 'Other Volatile Solvents, nec', 'Aromatic Hydrocarbons, nec', 'Propane', 
                 'Bromochlorodifluoromethane', 'Gamma-butyrolactone'],
    'misc': ['Pep pills', 'Miscellaneous Drugs', 'Aloin', 'Inadequately described', 'Anabolic Androgenic Steroids, nec', 'Levothyroxin sodium', 'Andriol', 'Petroleum', 'Stanazol', 'AAS', 'Genotropin', 
             'Anabolic Agents and Selected Hormones, nfd', 'Neo-Diophen', '1-(3,4-Methylenedioxybenzyl)piperazine', 'IGF-1', 'Other Anabolic Agents and Selected Hormones, nec', 'Actrapid',
             'Phenothiazines, nec', 'Lonavar', 'Butyl', 'Deca', 'Carbon tetrachloride', 'Miscellaneous Drugs '],
    'not stated': ['Not Stated', ' Not Stated', 'Actifed', 2, 5200, 5500, 9100, 3600, 4200, 6200, 1100, 1400, 3500, 4900, 6900, 5900, 7100, 2500, 1300, 6000, 2900, 3900,
                   1200, 9200, 9000, 2100]
}

In [ ]:
# and this is the 'bit extra'. Reverse mapping the dictionary so that the values are now keys.

inverse = {}
for k, v in drug_cat.items():
    for x in v: 
        inverse.setdefault(x, str(k))
        
# and replace the values in the original dataframe.
data = data.replace({"principal_drug_of_concern": inverse}) 

In [ ]:
# lets double check everything looks fine from this.
data['principal_drug_of_concern'].value_counts()

In [ ]:
# Need to change country of birth as inadequately described is listed twice

data['country_of_birth'].replace({'Inadequately described/Not stated': 'Inadequately Described/Not Stated'}, inplace = True) 
data['country_of_birth'].value_counts()

In [ ]:
# lets now categorise the treatment duration into some bins.
data['treatment_duration_days'] = pd.cut(data['treatment_duration_days'], bins = [0, 1, 10, 50, 100, 500, 6000],
                                                       labels = ['1', '1-10', '10-50', '50-100', '100-500', '200+'])
data['treatment_duration_days'].value_counts()


# Data exploration

In [ ]:
data.sort_values("_id")
single_id_data = data.copy()
single_id_data.drop_duplicates(subset = ['_id'], inplace = True)

plt.figure(figsize = (12, 6))
sns.countplot(x = 'age_group', data = data)
plt.title("Overall Frequency of Age Group")
plt.xlabel("Age Group")
plt.ylabel("Frequency")
plt.show()


So the sample is made up mostly of 20-49 year olds. 
Now any graphs need to take this into account. The next graph, looking at the amount of visits by age, will probably show a similar pattern.

In [ ]:
plt.figure(figsize = (15, 10))
sns.countplot(x = 'visit_number', hue = 'age_group', data = data)
plt.title("Visit Number by age group")
plt.xlabel("Visit Number")
plt.legend(title = 'Age Group')
plt.show()


Things don't change much here, there is a fairly uniform distribution in the amount of visits by age group. Perhaps the 30-39 and 40-49 categories show the biggest reduction

In [ ]:
age_grouped = data.copy()
age_grouped = age_grouped[['age_group', 'completed']]
age_group = age_grouped.groupby('age_group').mean()
age_group['completed'] = age_group['completed'] * 100
age_group['age_group'] = age_group.index

plt.figure(figsize= (12,6))
sns.barplot(x = 'age_group', y = 'completed', data = age_group)
plt.title('Proportion who completed treatment by age group')
plt.xlabel('Age Group')
plt.ylabel('% Completed Treatment')
plt.show()

It seems as though you are slightly less likely to finish treatment if you are 30-49 compared to the other age groups.

In [ ]:
treatment_type = data['main_treatment_type'].value_counts()
plt.figure(figsize = (12,6))
explode = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
plt.pie(treatment_type, labels = treatment_type.index, explode = explode)
plt.title("Distribution of treatment types")
plt.show();

Most patients got counselling, followed by info and education. Pharmacotherapy was the least required service.

In [ ]:
data['treatment_duration_days'].value_counts()

It looks like most individuals stayed for only one day. Lets see what they came in for.

In [ ]:
one_day_stay = data[data['treatment_duration_days'] == '1'].copy()
morethan_day_stay = data[data['treatment_duration_days'] != '1'].copy()

In [ ]:
stay_duration = data.copy()
stay_duration = stay_duration[['main_treatment_type', 'treatment_duration_days']]
stay_duration['treatment_duration_binned'] = data['treatment_duration_days'].apply(lambda x: '1' if x == '1' else '>1')

plt.figure(figsize = (12, 6))
sns.countplot(x = 'main_treatment_type', hue = 'treatment_duration_binned', data = stay_duration)
plt.title("Frequency of Treatment Types by length of stay")
plt.legend(title = 'Treatment Duration', labels = ['More than one day', 'One day'])
plt.xlabel('Treatment Type')
plt.ylabel('Frequency')
plt.xticks(rotation = 45)
plt.show()

Info and education was the reason most people only stayed one day. Others came in for an assessment only.
Counselling was the reason most people stayed more than one day.

The vast majority of people who stayed for more than one day stayed for counselling.
Perhaps we can see what it was that helped these people complete counselling

In [ ]:
counselling = morethan_day_stay.copy()
counselling = counselling[counselling['main_treatment_type'] == 'counselling']
counselling.head()

In [ ]:
plt.figure(figsize = (12,6))
sns.countplot(x = 'completed', hue = 'sector', data = counselling)
plt.title("Whether they completed counselling by sector")
plt.xticks(ticks = [0, 1], labels = ['No', 'Yes'])
plt.ylabel("Frequency")
plt.show()


It seems like the results from the private sector are a bit better

In [ ]:
print("The total number of people who recieved counselling: ", counselling['_id'].count())  
print("The total number of people who completed counselling: ", counselling['completed'].value_counts()[1])
print("The total number of people who came in more than once: ", counselling['more_than_one_visit'].value_counts()[1])


# Creating a model to see if we can predict whether people will complete counselling

In [ ]:
x = counselling.loc[:, counselling.columns != "completed"]
y = counselling.loc[:, counselling.columns == 'completed']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.25)

# splitting the data into training and testing

In [ ]:
x_train.head()

In [ ]:
# need to drop RfC grouping as this is the column that states completed, as well as some other variables

x_train.drop(columns = ['_id', 'rfc_grouping','month_of_commencement', 'month_of_cessation'], inplace = True)
x_test.drop(columns = ['_id', 'rfc_grouping','month_of_commencement', 'month_of_cessation'], inplace = True)

In [ ]:
# convert some last features to object.
object_features = ['start_year', 'start_month', 'end_year', 'end_month', 'principal_drug_of_concern', 'age_group', 'treatment_duration_days']
x_train[object_features] = x_train[object_features].astype('object')
x_test[object_features] = x_test[object_features].astype('object')

In [ ]:
# one hot encoding variables
cat_features = list(x_train.select_dtypes(include = ['object', 'category']).columns)
x_train = pd.get_dummies(x_train, columns = cat_features, dtype = 'int')
x_test = pd.get_dummies(x_test, columns = cat_features, dtype = 'int')

In [ ]:
# one last check.
x_train.info()

Create a model

will just build a simple logistic regression.

In [ ]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
# fill any potential missing categories and make sure dataframes are the same size.
x_train, x_test = x_train.align(x_test, join = "outer", axis = 1)

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [ ]:
logr = LogisticRegression(max_iter = 1000)

scores = cross_val_score(logr, x_train, y_train.values.ravel(), scoring = 'accuracy', cv = 10)
print(scores.mean())

In [ ]:
logr.fit(x_train, y_train.values.ravel())
predictions = logr.predict(x_test)
accuracy_score(y_test, predictions)

The initial model, a logistic regression with no hyperparameter tuning has an accuracy of 63.3%

In [ ]:
# Now we'll try a random a random forest classifier

rf_classifier = RandomForestClassifier()

rf_classifier.fit(x_train, y_train.values.ravel())
predicted = rf_classifier.predict(x_test)

accuracy_score(y_test, predicted)

Slightly better accuracy with the random forest classifier

In [ ]:
# better initial accuracy on this so we will do some simple hyperparameter tuning.

param_grid = [
    {'n_estimators': [50, 100, 200], 'max_depth': [15, 20, 40]}
]

grid_search = GridSearchCV(rf_classifier, param_grid, cv = 5, scoring = 'accuracy', return_train_score = True)
grid_search.fit(x_train, y_train.values.ravel())

grid_search.best_params_

In [ ]:
final_model = grid_search.best_estimator_

final_predictions = final_model.predict(x_test)
accuracy_score(final_predictions, y_test)


Not really any better than the original. It could simply be that there is no disercnible relationhship for ML to make use of, or that I need to do more feature engineering.